****TensorFlow Convolution Layer****

Let's examine how to implement a CNN in TensorFlow.

TensorFlow provides the tf.nn.conv2d() and tf.nn.bias_add() functions to create your own convolutional layers.

In [3]:
# import tensorflow as tf

# Tensorflow 1 compatibility
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Output depth
k_output = 64

# Image Properties
image_width = 10
image_height = 10
color_channels = 3

# Convolution filter
filter_size_width = 5
filter_size_height = 5

# Input/Image
input = tf.placeholder(
    tf.float32,
    shape=[None, image_height, image_width, color_channels])

# Weight and bias
weight = tf.Variable(tf.truncated_normal(
    [filter_size_height, filter_size_width, color_channels, k_output]))
bias = tf.Variable(tf.zeros(k_output))

# Apply Convolution
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
print(conv_layer)
# Add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
print(conv_layer)
# Apply activation function
conv_layer = tf.nn.relu(conv_layer)

print(conv_layer)

# Apply Max Pooling
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')

print(conv_layer)


Tensor("Conv2D_1:0", shape=(?, 5, 5, 64), dtype=float32)
Tensor("BiasAdd_1:0", shape=(?, 5, 5, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 5, 5, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 3, 3, 64), dtype=float32)


****Convolutional Network in TensorFlow****

It's time to walk through an example Convolutional Neural Network (CNN) in TensorFlow. 

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

# import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units



# Weights and Biases

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}



# Convolutions

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)



# Max Pooling 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')



# CNN 

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out



# Session

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch  1, Batch   1 -Loss: 62195.6172 Validation Accuracy: 0.179688
Epoch  1, Batch   2 -Loss: 47528.6562 Validation Accuracy: 0.191406
Epoch  1, Batch   3 -Loss: 36714.1328 Validation Accuracy: 0.203125
Epoch  1, Batch   4 -Loss: 26600.4141 Validation Accuracy: 0.199219
Epoch  1, Batch   5 -Loss: 22554.4707 Validation Accuracy: 0.226562
Epoch  1, Batch   6 -Loss: 22382.2363 Validation Accuracy: 0.257812
Epoch  1, Batch   7 -Loss: 22327.7617 Validation Accuracy: 0.261719
Epoch  1, Batch   8 -Loss: 20430.7637 Validation Accuracy: 0

Epoch  1, Batch 112 -Loss:  3841.2607 Validation Accuracy: 0.582031
Epoch  1, Batch 113 -Loss:  4013.3804 Validation Accuracy: 0.582031
Epoch  1, Batch 114 -Loss:  4410.1240 Validation Accuracy: 0.589844
Epoch  1, Batch 115 -Loss:  4969.1484 Validation Accuracy: 0.589844
Epoch  1, Batch 116 -Loss:  4492.6641 Validation Accuracy: 0.593750
Epoch  1, Batch 117 -Loss:  3779.3723 Validation Accuracy: 0.593750
Epoch  1, Batch 118 -Loss:  3735.6086 Validation Accuracy: 0.601562
Epoch  1, Batch 119 -Loss:  4725.3682 Validation Accuracy: 0.597656
Epoch  1, Batch 120 -Loss:  3539.8950 Validation Accuracy: 0.605469
Epoch  1, Batch 121 -Loss:  5028.6846 Validation Accuracy: 0.601562
Epoch  1, Batch 122 -Loss:  4888.5518 Validation Accuracy: 0.597656
Epoch  1, Batch 123 -Loss:  3669.9377 Validation Accuracy: 0.601562
Epoch  1, Batch 124 -Loss:  4214.2949 Validation Accuracy: 0.609375
Epoch  1, Batch 125 -Loss:  4621.0283 Validation Accuracy: 0.609375
Epoch  1, Batch 126 -Loss:  4808.9824 Validation

Epoch  1, Batch 235 -Loss:  2214.8411 Validation Accuracy: 0.691406
Epoch  1, Batch 236 -Loss:  3217.7026 Validation Accuracy: 0.675781
Epoch  1, Batch 237 -Loss:  1885.9535 Validation Accuracy: 0.683594
Epoch  1, Batch 238 -Loss:  2354.3052 Validation Accuracy: 0.683594
Epoch  1, Batch 239 -Loss:  2151.4795 Validation Accuracy: 0.675781
Epoch  1, Batch 240 -Loss:  2506.5869 Validation Accuracy: 0.679688
Epoch  1, Batch 241 -Loss:  1911.1775 Validation Accuracy: 0.679688
Epoch  1, Batch 242 -Loss:  1545.4371 Validation Accuracy: 0.683594
Epoch  1, Batch 243 -Loss:  3103.3809 Validation Accuracy: 0.687500
Epoch  1, Batch 244 -Loss:  2998.9517 Validation Accuracy: 0.691406
Epoch  1, Batch 245 -Loss:  2126.9617 Validation Accuracy: 0.691406
Epoch  1, Batch 246 -Loss:  1947.1040 Validation Accuracy: 0.687500
Epoch  1, Batch 247 -Loss:  1772.0435 Validation Accuracy: 0.687500
Epoch  1, Batch 248 -Loss:  2171.7275 Validation Accuracy: 0.695312
Epoch  1, Batch 249 -Loss:  2571.4004 Validation

Epoch  1, Batch 356 -Loss:  1691.6726 Validation Accuracy: 0.726562
Epoch  1, Batch 357 -Loss:  1710.2498 Validation Accuracy: 0.730469
Epoch  1, Batch 358 -Loss:  1377.9390 Validation Accuracy: 0.722656
Epoch  1, Batch 359 -Loss:  1960.5575 Validation Accuracy: 0.726562
Epoch  1, Batch 360 -Loss:  1378.6381 Validation Accuracy: 0.718750
Epoch  1, Batch 361 -Loss:  1298.4897 Validation Accuracy: 0.726562
Epoch  1, Batch 362 -Loss:  1761.2302 Validation Accuracy: 0.722656
Epoch  1, Batch 363 -Loss:  1174.2400 Validation Accuracy: 0.726562
Epoch  1, Batch 364 -Loss:  1575.9923 Validation Accuracy: 0.726562
Epoch  1, Batch 365 -Loss:  1438.6218 Validation Accuracy: 0.726562
Epoch  1, Batch 366 -Loss:  2583.7292 Validation Accuracy: 0.726562
Epoch  1, Batch 367 -Loss:  2249.8096 Validation Accuracy: 0.734375
Epoch  1, Batch 368 -Loss:  2223.5540 Validation Accuracy: 0.726562
Epoch  1, Batch 369 -Loss:  1765.7310 Validation Accuracy: 0.730469
Epoch  1, Batch 370 -Loss:  1471.2979 Validation

Epoch  2, Batch  48 -Loss:  1077.2748 Validation Accuracy: 0.742188
Epoch  2, Batch  49 -Loss:  2703.6714 Validation Accuracy: 0.742188
Epoch  2, Batch  50 -Loss:  1692.1519 Validation Accuracy: 0.742188
Epoch  2, Batch  51 -Loss:  1034.7422 Validation Accuracy: 0.742188
Epoch  2, Batch  52 -Loss:  1969.9011 Validation Accuracy: 0.742188
Epoch  2, Batch  53 -Loss:  1688.0610 Validation Accuracy: 0.738281
Epoch  2, Batch  54 -Loss:  1821.2762 Validation Accuracy: 0.734375
Epoch  2, Batch  55 -Loss:  1938.0037 Validation Accuracy: 0.738281
Epoch  2, Batch  56 -Loss:  1358.2260 Validation Accuracy: 0.746094
Epoch  2, Batch  57 -Loss:  1524.9851 Validation Accuracy: 0.750000
Epoch  2, Batch  58 -Loss:  1253.5220 Validation Accuracy: 0.746094
Epoch  2, Batch  59 -Loss:  1952.6187 Validation Accuracy: 0.750000
Epoch  2, Batch  60 -Loss:  1697.4802 Validation Accuracy: 0.757812
Epoch  2, Batch  61 -Loss:  1656.1188 Validation Accuracy: 0.757812
Epoch  2, Batch  62 -Loss:   984.9797 Validation

...

Epoch  9, Batch 352 -Loss:   291.5649 Validation Accuracy: 0.820312
Epoch  9, Batch 353 -Loss:   231.0876 Validation Accuracy: 0.820312
Epoch  9, Batch 354 -Loss:   339.0758 Validation Accuracy: 0.820312
Epoch  9, Batch 355 -Loss:   273.4266 Validation Accuracy: 0.820312
Epoch  9, Batch 356 -Loss:   492.2883 Validation Accuracy: 0.824219
Epoch  9, Batch 357 -Loss:   253.1982 Validation Accuracy: 0.820312
Epoch  9, Batch 358 -Loss:   231.7174 Validation Accuracy: 0.820312
Epoch  9, Batch 359 -Loss:    88.7326 Validation Accuracy: 0.824219
Epoch  9, Batch 360 -Loss:   229.2651 Validation Accuracy: 0.820312
Epoch  9, Batch 361 -Loss:   254.7954 Validation Accuracy: 0.824219
Epoch  9, Batch 362 -Loss:   399.2161 Validation Accuracy: 0.824219
Epoch  9, Batch 363 -Loss:   427.5017 Validation Accuracy: 0.824219
Epoch  9, Batch 364 -Loss:   285.8650 Validation Accuracy: 0.824219
Epoch  9, Batch 365 -Loss:   385.0230 Validation Accuracy: 0.824219
Epoch  9, Batch 366 -Loss:   271.3227 Validation

Epoch 10, Batch  44 -Loss:   358.5461 Validation Accuracy: 0.812500
Epoch 10, Batch  45 -Loss:   342.7106 Validation Accuracy: 0.812500
Epoch 10, Batch  46 -Loss:   430.3949 Validation Accuracy: 0.812500
Epoch 10, Batch  47 -Loss:   271.7530 Validation Accuracy: 0.812500
Epoch 10, Batch  48 -Loss:   306.0511 Validation Accuracy: 0.812500
Epoch 10, Batch  49 -Loss:   280.5281 Validation Accuracy: 0.816406
Epoch 10, Batch  50 -Loss:   347.3112 Validation Accuracy: 0.816406
Epoch 10, Batch  51 -Loss:   586.5555 Validation Accuracy: 0.812500
Epoch 10, Batch  52 -Loss:   313.8545 Validation Accuracy: 0.812500
Epoch 10, Batch  53 -Loss:   430.6371 Validation Accuracy: 0.816406
Epoch 10, Batch  54 -Loss:   370.6367 Validation Accuracy: 0.820312
Epoch 10, Batch  55 -Loss:   237.2984 Validation Accuracy: 0.812500
Epoch 10, Batch  56 -Loss:   219.0490 Validation Accuracy: 0.816406
Epoch 10, Batch  57 -Loss:   360.8006 Validation Accuracy: 0.816406
Epoch 10, Batch  58 -Loss:   291.9670 Validation

Epoch 10, Batch 167 -Loss:   258.8071 Validation Accuracy: 0.812500
Epoch 10, Batch 168 -Loss:   254.1250 Validation Accuracy: 0.812500
Epoch 10, Batch 169 -Loss:   414.9463 Validation Accuracy: 0.812500
Epoch 10, Batch 170 -Loss:   387.5364 Validation Accuracy: 0.812500
Epoch 10, Batch 171 -Loss:   216.9044 Validation Accuracy: 0.812500
Epoch 10, Batch 172 -Loss:   442.8883 Validation Accuracy: 0.816406
Epoch 10, Batch 173 -Loss:   284.0491 Validation Accuracy: 0.816406
Epoch 10, Batch 174 -Loss:   273.7452 Validation Accuracy: 0.816406
Epoch 10, Batch 175 -Loss:   247.0789 Validation Accuracy: 0.812500
Epoch 10, Batch 176 -Loss:   296.7942 Validation Accuracy: 0.816406
Epoch 10, Batch 177 -Loss:   459.0102 Validation Accuracy: 0.816406
Epoch 10, Batch 178 -Loss:   284.9625 Validation Accuracy: 0.812500
Epoch 10, Batch 179 -Loss:   324.6035 Validation Accuracy: 0.816406
Epoch 10, Batch 180 -Loss:   150.9251 Validation Accuracy: 0.812500
Epoch 10, Batch 181 -Loss:   407.9738 Validation

Epoch 10, Batch 290 -Loss:   370.1136 Validation Accuracy: 0.820312
Epoch 10, Batch 291 -Loss:   205.1526 Validation Accuracy: 0.820312
Epoch 10, Batch 292 -Loss:   356.7952 Validation Accuracy: 0.820312
Epoch 10, Batch 293 -Loss:   176.0357 Validation Accuracy: 0.816406
Epoch 10, Batch 294 -Loss:   335.0289 Validation Accuracy: 0.820312
Epoch 10, Batch 295 -Loss:   413.5162 Validation Accuracy: 0.820312
Epoch 10, Batch 296 -Loss:   334.3896 Validation Accuracy: 0.820312
Epoch 10, Batch 297 -Loss:   330.1435 Validation Accuracy: 0.820312
Epoch 10, Batch 298 -Loss:   202.6145 Validation Accuracy: 0.820312
Epoch 10, Batch 299 -Loss:   469.7461 Validation Accuracy: 0.820312
Epoch 10, Batch 300 -Loss:   259.8979 Validation Accuracy: 0.820312
Epoch 10, Batch 301 -Loss:   386.9987 Validation Accuracy: 0.820312
Epoch 10, Batch 302 -Loss:   310.2915 Validation Accuracy: 0.820312
Epoch 10, Batch 303 -Loss:   446.2789 Validation Accuracy: 0.820312
Epoch 10, Batch 304 -Loss:   427.4270 Validation

Epoch 10, Batch 413 -Loss:   217.3914 Validation Accuracy: 0.820312
Epoch 10, Batch 414 -Loss:   420.0642 Validation Accuracy: 0.820312
Epoch 10, Batch 415 -Loss:   171.6143 Validation Accuracy: 0.824219
Epoch 10, Batch 416 -Loss:   269.3426 Validation Accuracy: 0.828125
Epoch 10, Batch 417 -Loss:   212.6116 Validation Accuracy: 0.820312
Epoch 10, Batch 418 -Loss:   266.6682 Validation Accuracy: 0.820312
Epoch 10, Batch 419 -Loss:   302.2051 Validation Accuracy: 0.820312
Epoch 10, Batch 420 -Loss:   295.9503 Validation Accuracy: 0.820312
Epoch 10, Batch 421 -Loss:   512.1848 Validation Accuracy: 0.820312
Epoch 10, Batch 422 -Loss:   211.6385 Validation Accuracy: 0.820312
Epoch 10, Batch 423 -Loss:   248.4650 Validation Accuracy: 0.820312
Epoch 10, Batch 424 -Loss:   212.9218 Validation Accuracy: 0.820312
Epoch 10, Batch 425 -Loss:   182.5101 Validation Accuracy: 0.820312
Epoch 10, Batch 426 -Loss:   412.1809 Validation Accuracy: 0.820312
Epoch 10, Batch 427 -Loss:   378.4854 Validation